# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jetpur,21.7333,70.6167,79.72,83,98,11.01,IN,1722996809
1,1,touros,-5.1989,-35.4608,74.46,84,1,12.91,BR,1722996810
2,2,colares,38.7992,-9.4469,70.79,83,30,17.90,PT,1722996811
3,3,chonchi,-42.6128,-73.8092,38.12,96,10,1.61,CL,1722996812
4,4,olonkinbyen,70.9221,-8.7187,42.03,100,100,13.62,SJ,1722996813


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# %pip install cartopy geoviews pyproj


# Configure the map plot
city_map=city_data_df.hvplot.scatter(
   "Lng", 
   "Lat", 
    color = "City",
    alpha = 0.6,
    size = "Humidity",
    tiles = "OSM"
).opts(width = 800, height = 500, tools = ['hover'], title = "City Map"

)

# Display the map
city_map


:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] > 10)]
# Drop any rows with null values
weather_filtered_df = weather_filtered_df.dropna()

# Display sample data
weather_filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,bethel,41.3712,-73.4140,70.90,89,100,1.01,US,1722996574
14,14,bengkulu,-3.8004,102.2655,79.05,82,72,5.17,ID,1722996824
44,44,kodinsk,58.6881,99.1844,71.29,71,81,5.95,RU,1722996859
47,47,holualoa,19.6228,-155.9522,76.84,86,75,4.61,US,1722996862
48,48,port mathurin,-19.6833,63.4167,72.41,73,48,8.63,MU,1722996863


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,bethel,US,41.3712,-73.4140,89,
14,bengkulu,ID,-3.8004,102.2655,82,
44,kodinsk,RU,58.6881,99.1844,71,
47,holualoa,US,19.6228,-155.9522,86,
48,port mathurin,MU,-19.6833,63.4167,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: No hotel found
bengkulu - nearest hotel: No hotel found
kodinsk - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
mitu - nearest hotel: No hotel found
simplicio mendes - nearest hotel: No hotel found
gunjur - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
porto novo - nearest hotel: No hotel found
novoazovs'k - nearest hotel: No hotel found
praia da vitoria - nearest hotel: No hotel found
impfondo - nearest hotel: No hotel found
college - nearest hotel: No hotel found
luang namtha - nearest hotel: No hotel found
yagoua - nearest hotel: No hotel found
inongo - nearest hotel: No hotel found
lodhikheda - nearest hotel: No hotel found
peace river - nearest hotel: No hotel found
zeya - nearest hotel: No hotel found
fo

,City,Country,Lat,Lng,Humidity,Hotel Name
12,bethel,US,41.3712,-73.4140,89,No hotel found
14,bengkulu,ID,-3.8004,102.2655,82,No hotel found
44,kodinsk,RU,58.6881,99.1844,71,No hotel found
47,holualoa,US,19.6228,-155.9522,86,No hotel found
48,port mathurin,MU,-19.6833,63.4167,73,No hotel found
...,...,...,...,...,...,...
523,siraha,NP,26.6541,86.2087,93,No hotel found
536,maragogi,BR,-9.0122,-35.2225,82,No hotel found
539,nicoya,CR,10.1483,-85.4520,97,No hotel found
542,mandya,IN,12.5242,76.8958,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.scatter(
   "Lng", 
   "Lat", 
    color = "Hotel Name",
    alpha = 0.6,
    size = "Humidity",
    tiles = "OSM"
).opts(width = 800, height = 500, tools = ['hover'], title = "Hotel Map"

)

# Display the map
hotel_map

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,Hotel Name,Humidity)